# Introduction

In the last tutorial, we learned how to select relevant data out of a DataFrame or Series. Plucking the right data out of our data representation is critical to getting work done, as we demonstrated in the exercises.

However, the data does not always come out of memory in the format we want it in right out of the bat. Sometimes we have to do some more work ourselves to reformat it for the task at hand.  This tutorial will cover different operations we can apply to our data to get the input "just right". 


In [29]:

import pandas as pd
pd.set_option('max_rows', 5)
import numpy as np
data = pd.read_csv('Salary_Data.csv')

In [30]:
data

,YearsExperience,Salary,Skill,Age,education,expense,savings
0,100.0,3900,c++,22,bachelor,31200.0,7800.0
1,110.0,390000,python,22,masters,31200.0,7800.0
...,...,...,...,...,...,...,...
28,10.3,122391,python,26,masters,97912.8,24478.2
29,10.5,121872,c++,27,bachelor,97497.6,24374.4


# Summary functions

Pandas provides many simple "summary functions" (not an official name) which restructure the data in some useful way. For example, consider the `describe()` method:

In [31]:
data.describe()

,YearsExperience,Salary,Age,expense,savings
count,30.000000,30.0,30.000000,30.00,30.00
mean,12.136667,86281.4,24.833333,60601.12,15150.28
...,...,...,...,...,...
75%,8.575000,104512.0,27.000000,80435.80,20108.95
max,110.000000,390000.0,29.000000,97912.80,24478.20


This method generates a high-level summary of the attributes of the given column. It is type-aware, meaning that its output changes based on the data type of the input. The output above only makes sense for numerical data; for string data here's what we get:

In [32]:
data.education.describe()

count           30
unique           2
top       bachelor
freq            20
Name: education, dtype: object

If you want to get some particular simple summary statistic about a column in a DataFrame or a Series, there is usually a helpful pandas function that makes it happen. 

For example, to see the mean of the points allotted (e.g. how well an averagely rated wine does), we can use the `mean()` function:

In [33]:
data.Salary.mean()

86281.4

To see a list of unique values we can use the `unique()` function:

In [34]:
data.Salary.unique()

array([  3900, 390000,  37731,  43525,  39891,  56642,  60150,  54445,
        64445,  57189,  63218,  55794,  56957,  57081,  61111,  67938,
        66029,  83088,  81363,  93940,  91738,  98273, 101302, 113812,
       109431, 105582, 116969, 112635, 122391, 121872], dtype=int64)

To see a list of unique values _and_ how often they occur in the dataset, we can use the `value_counts()` method:

In [35]:
data.education.value_counts()

bachelor    20
masters     10
Name: education, dtype: int64

# Maps

A **map** is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. In data science we often have a need for creating new representations from existing data, or for transforming data from the format it is in now to the format that we want it to be in later. Maps are what handle this work, making them extremely important for getting your work done!

There are two mapping methods that you will use often. 

[`map()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) is the first, and slightly simpler one. For example, suppose that we wanted to remean the scores the salary received to 0. We can do this as follows:

In [36]:
salary_mean = data.Salary.mean()
data.Salary.map(lambda p: p - salary_mean)

0     -82381.4
1     303718.6
        ...   
28     36109.6
29     35590.6
Name: Salary, Length: 30, dtype: float64

The function you pass to `map()` should expect a single value from the Series (a point value, in the above example), and return a transformed version of that value. `map()` returns a new Series where all the values have been transformed by your function.

[`apply()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html) is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.

In [37]:
def remean_salary(row):
    row.Salary = row.Salary - salary_mean
    return row

data.apply(remean_salary, axis='columns')

,YearsExperience,Salary,Skill,Age,education,expense,savings
0,100.0,-82381.4,c++,22,bachelor,31200.0,7800.0
1,110.0,303718.6,python,22,masters,31200.0,7800.0
...,...,...,...,...,...,...,...
28,10.3,36109.6,python,26,masters,97912.8,24478.2
29,10.5,35590.6,c++,27,bachelor,97497.6,24374.4


If we had called `data.apply()` with `axis='index'`, then instead of passing a function to transform each row, we would need to give a function to transform each *column*.

Note that `map()` and `apply()` return new, transformed Series and DataFrames, respectively. They don't modify the original data they're called on. If we look at the first row of `data`, we can see that it still has its original `points` value.

In [38]:
data.head(1)

,YearsExperience,Salary,Skill,Age,education,expense,savings
0,100.0,3900,c++,22,bachelor,31200.0,7800.0


Pandas provides many common mapping operations as built-ins. For example, here's a faster way of remeaning our salary column:

In [39]:
salary_mean = data.Salary.mean()
data.Salary - salary_mean

0     -82381.4
1     303718.6
        ...   
28     36109.6
29     35590.6
Name: Salary, Length: 30, dtype: float64

In this code we are performing an operation between a lot of values on the left-hand side (everything in the Series) and a single value on the right-hand side (the mean value). Pandas looks at this expression and figures out that we must mean to subtract that mean value from every value in the dataset.

Pandas will also understand what to do if we perform these operations between Series of equal length. 

In [40]:
data.education + " - " + data.Skill

0       bachelor - c++
1     masters - python
            ...       
28    masters - python
29      bachelor - c++
Length: 30, dtype: object

These operators are faster than `map()` or `apply()` because they uses speed ups built into pandas. All of the standard Python operators (`>`, `<`, `==`, and so on) work in this manner.

However, they are not as flexible as `map()` or `apply()`, which can do more advanced things, like applying conditional logic, which cannot be done with addition and subtraction alone.

